# Data Analysis - Irradiation of a Fibre Optic Sensor
Measuring humidity in radioactive environments is challenging because - as you might expect - the radiation is massively damaging to sensors. Most commercial humidity sensors are electronic (HIH??), get wiped out by even a small dose of radiation (how much??).

This project will take you through the 